<td>
   <a target="_blank" href="https://www.clarifai.com/" ><img src="https://upload.wikimedia.org/wikipedia/commons/b/bc/Clarifai_Logo_FC_Web.png" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Clarifai/examples/blob/main/models/model_train/image-segmentation_training.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab"></a>
</td>

# Models



Clarifai offers a range of powerful model types, each designed to generate meaningful outputs based on user specific inputs and AI tasks.

There are wide variety of models that can be used as standalone solutions, or as building blocks for your own custom business solutions.
 

Clarifai Models are the recommended starting points for many users because they offer incredibly fast training times when you customize them using the "embedding-classifier" (Transfer Learning Classifier) model type.

But there are many cases where accuracy and the ability to carefully target solutions take priority over speed and ease of use. Additionally, you may need a model to learn new features, not recognized by existing Clarifai Models. For these cases, it is possible to "deep fine-tune" your custom models and integrate them directly within your workflows.

You might consider deep training if you have:

- A custom tailored dataset
- Accurate labels
- Expertise and time to fine-tune models

_______
On the [Clarifai Community](https://clarifai.com/explore) explore page, you can click the [Models](https://clarifai.com/explore/models) tab to search and access the models available for everyone to use.

This notebook contains Model Train demo for **visual-segmenter** Model Type with **MMDetection** Template.

# Getting Started

### Installation

In [ ]:
! pip install clarifai'[all]'

In [1]:
import os
os.environ["CLARIFAI_PAT"] = "PAT" # replace with your own PAT key here

*Note: Guide to get your [PAT](https://docs.clarifai.com/clarifai-basics/authentication/personal-access-tokens)*

### For Colab
To access data files from Clarifai examples repo, you can clone the repo

In [ ]:
!git clone https://github.com/Clarifai/examples.git
%cd /content/examples/models/model_train

### For Local

In [ ]:
git clone https://github.com/Clarifai/examples.git
cd examples/models/model_train

## VISUAL-SEGMENTER

**Input: Images and videos**

**Output: Regions**

Visual Segmenter, also known as Semantic Segmentation, is a type of deep fine-tuned model used in image analysis and understanding tasks.

It aims to achieve a fine-grained understanding of the content within an image by associating each pixel with a particular class label. This is more detailed than traditional object detection, which typically identifies bounding boxes around objects.

### Creating an App

In [2]:
from clarifai.client.user import User
#replace your "user_id"
client = User(user_id="user_id")

In [3]:
app = client.create_app(app_id="app_id", base_workflow="Universal")

2024-03-20 15:11:55 INFO     clarifai.client.user:                                                      ]8;id=977135;file:///Users/adithyansukumar/.pyenv/versions/3.9.10/lib/python3.9/site-packages/clarifai/client/user.py\user.py]8;;\:]8;id=812141;file:///Users/adithyansukumar/.pyenv/versions/3.9.10/lib/python3.9/site-packages/clarifai/client/user.py#127\127]8;;\
                             App created                                                                           
                             code: SUCCESS                                                                         
                             description: "Ok"                                                                     
                             req_id: "69762f1acc47b1b9f14cca43a67ff741"                                            
                                                                                                                   

### Uploading Image Segmentation Dataset

In [4]:
module_path = os.path.join(os.getcwd().split('/models/model_train')[0],'datasets/upload/image_segmentation/coco')
module_path

'/Users/adithyansukumar/work/ml_training_error/examples/datasets/upload/image_segmentation/coco'

In [5]:
#importing load_module_dataloader for calling the dataloader object in dataset.py in the local data folder
from clarifai.datasets.upload.utils import load_module_dataloader

In [6]:
coco_dataloader = load_module_dataloader(module_path)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [7]:
dataset = app.create_dataset(dataset_id="segmentation_dataset")
dataset.upload_dataset(dataloader=coco_dataloader)

2024-03-20 15:12:01 INFO     clarifai.client.app:                                                        ]8;id=480330;file:///Users/adithyansukumar/.pyenv/versions/3.9.10/lib/python3.9/site-packages/clarifai/client/app.py\app.py]8;;\:]8;id=177524;file:///Users/adithyansukumar/.pyenv/versions/3.9.10/lib/python3.9/site-packages/clarifai/client/app.py#310\310]8;;\
                             Dataset created                                                                       
                             code: SUCCESS                                                                         
                             description: "Ok"                                                                     
                             req_id: "d44173ad561e47bb435f92d564b70d5e"                                            
                                                                                                                   

Uploading Dataset: 100%|██████████| 1/1 [01:04<00:00, 64.69s/it]


### List Trainable Model Types

In [7]:
app.list_trainable_model_types()

['visual-classifier',
 'visual-detector',
 'visual-segmenter',
 'visual-embedder',
 'clusterer',
 'text-classifier',
 'embedding-classifier',
 'text-to-text']

### Create a Model

In [8]:
MODEL_ID = "segmenter"
MODEL_TYPE_ID = "visual-segmenter"
model = app.create_model(model_id=MODEL_ID, model_type_id=MODEL_TYPE_ID)

2024-03-20 15:13:06 INFO     clarifai.client.app:                                                        ]8;id=762214;file:///Users/adithyansukumar/.pyenv/versions/3.9.10/lib/python3.9/site-packages/clarifai/client/app.py\app.py]8;;\:]8;id=379523;file:///Users/adithyansukumar/.pyenv/versions/3.9.10/lib/python3.9/site-packages/clarifai/client/app.py#334\334]8;;\
                             Model created                                                                         
                             code: SUCCESS                                                                         
                             description: "Ok"                                                                     
                             req_id: "38ca9570013f8135372a318da9e4f28e"                                            
                                                                                                                   

### List Templates for the Model Type

Templates give you the control to choose the specific architecture used by your neural network, as well as define a set of hyperparameters you can use to fine-tune the way your model learns.

In [9]:
model.list_training_templates()

['MMSegmentation_AdvancedConfig', 'MMSegmentation_SegFormer']

### Save params
Save the parameters for the specific model template in a YAML file

In [10]:
model_params = model.get_params(template='MMSegmentation_SegFormer')

In [14]:
import yaml
# Preview YAML content
YAML_FILE_PATH="/Users/adithyansukumar/work/ml_training_error/examples/models/model_train/params.yaml"
file = open(YAML_FILE_PATH)
data = yaml.safe_load(file)
data

{'dataset_id': '',
 'dataset_version_id': '',
 'concepts': [],
 'train_params': {'invalid_data_tolerance_percent': 5.0,
  'template': 'MMSegmentation_SegFormer',
  'image_size': [520.0],
  'batch_size': 2.0,
  'num_epochs': 1.0,
  'per_item_lrate': 7.5e-06,
  'pretrained_weights': 'ade20k'}}

#### Get param info

In [15]:
print(model.get_param_info(param = 'pretrained_weights'))

{'fieldType': 'ENUM', 'defaultValue': 'ade20k', 'description': 'whether to init with pretrained weights.', 'modelTypeEnumOptions': [{'id': 'None'}, {'id': 'ade20k'}], 'param': 'pretrained_weights'}


### Update params
Note: User can edit the params in the YAML file or directly update the params using model.update_params()

In [16]:
#list concepts in the app
concepts = [concept.id for concept in app.list_concepts()]
concepts

['id-elephant',
 'id-laptop',
 'id-diningtable',
 'id-cellphone',
 'id-cat',
 'id-keyboard',
 'id-mouse',
 'id-toilet',
 'id-oven',
 'id-pizza',
 'id-clock',
 'id-chair',
 'id-cup',
 'id-kite',
 'id-couch',
 'id-baseballbat',
 'id-handbag',
 'id-person']

In [17]:
#creating dataset version
dataset_version = dataset.create_version()
dataset_version_id = dataset_version.version.id
dataset_version_id

2024-03-20 15:15:30 INFO     clarifai.client.dataset:                                                ]8;id=455037;file:///Users/adithyansukumar/.pyenv/versions/3.9.10/lib/python3.9/site-packages/clarifai/client/dataset.py\dataset.py]8;;\:]8;id=608924;file:///Users/adithyansukumar/.pyenv/versions/3.9.10/lib/python3.9/site-packages/clarifai/client/dataset.py#101\101]8;;\
                             Dataset Version created                                                               
                             code: SUCCESS                                                                         
                             description: "Ok"                                                                     
                             req_id: "6f611476ff874a0889fc6f0b974b50ac"                                            
                                                                                                                   

'd9b62cac13a2480c819b8341d73794ca'

In [18]:
#update params
model.update_params(dataset_id = 'segmentation_dataset', dataset_version_id=dataset_version_id,
                    concepts = concepts, num_epochs = 5)

In [19]:
model.training_params

{'dataset_id': 'segmentation_dataset',
 'dataset_version_id': 'd9b62cac13a2480c819b8341d73794ca',
 'concepts': ['id-elephant',
  'id-laptop',
  'id-diningtable',
  'id-cellphone',
  'id-cat',
  'id-keyboard',
  'id-mouse',
  'id-toilet',
  'id-oven',
  'id-pizza',
  'id-clock',
  'id-chair',
  'id-cup',
  'id-kite',
  'id-couch',
  'id-baseballbat',
  'id-handbag',
  'id-person'],
 'train_params': {'invalid_data_tolerance_percent': 5.0,
  'template': 'MMSegmentation_SegFormer',
  'image_size': [520.0],
  'batch_size': 2.0,
  'num_epochs': 5,
  'per_item_lrate': 7.5e-06,
  'pretrained_weights': 'ade20k'}}

### Start Model Training

In [20]:
model_version_id = model.train()

2024-03-20 15:15:37 INFO     clarifai.client.model:                                                    ]8;id=734384;file:///Users/adithyansukumar/.pyenv/versions/3.9.10/lib/python3.9/site-packages/clarifai/client/model.py\model.py]8;;\:]8;id=75903;file:///Users/adithyansukumar/.pyenv/versions/3.9.10/lib/python3.9/site-packages/clarifai/client/model.py#241\241]8;;\
                             Model Training Started                                                                
                             code: SUCCESS                                                                         
                             description: "Ok"                                                                     
                             req_id: "0db699c6f5b7bd9890f82def15bd78dc"                                            
                                                                                                                   

### Check Model Training Status

In [21]:
import time
while True:
    status = model.training_status(version_id=model_version_id,training_logs=False)
    if status.code == 21106: #MODEL_TRAINING_FAILED
        print(status)
        break
    elif status.code == 21100: #MODEL_TRAINED
        print(status)
        break
    else:
        print("Current Status:",status)
        print("Waiting---")
        time.sleep(120)

Current Status: code: MODEL_QUEUED_FOR_TRAINING
description: "Model is currently in queue for training."

Waiting---
Current Status: code: MODEL_TRAINING
description: "Training stage in progress: 0/1 complete."

Waiting---
Current Status: code: MODEL_TRAINING
description: "Deployment stage in progress: 3/4 complete."

Waiting---
Current Status: code: MODEL_TRAINING
description: "Deployment stage in progress: 3/4 complete."

Waiting---
code: MODEL_TRAINED
description: "Model is trained and ready"



### Model Prediction
Predicting with the Trained Model.
Note: Refer this [notebook](https://github.com/Clarifai/examples/blob/main/models/model_predict.ipynb) for more info on Model Predict

In [22]:
IMAGE_PATH = os.path.join(os.getcwd().split('/models')[0],'datasets/upload/image_segmentation/coco/images/000000166563.jpg')

prediction_response = model.predict_by_filepath(IMAGE_PATH, input_type="image")

# Get the output
regions = prediction_response.outputs[0].data.regions

In [ ]:
# Display the predicted masks
import cv2
from urllib.request import urlopen
import numpy as np
from PIL import Image as PILImage
from io import BytesIO
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

img = plt.imread(IMAGE_PATH)
masks = []
concepts = []
for region in regions:
    if region.data.concepts[0].value > 0.05:
        masks.append(np.array(PILImage.open(BytesIO(region.region_info.mask.image.base64))))
        concepts.append(region.data.concepts[0].name)


# Generate random colors
colors = []
for i in range(len(masks)):
    r = random.randint(0,255)
    g = random.randint(0,255)
    b = random.randint(0,255)
    colors.append((b,g,r))

# Map masks to overlays
overlays = []
for i in range(len(masks)):
    mask = masks[i]
    color = colors[i]

    overlay = np.zeros_like(img)
    overlay[mask > 0] = color
    overlays.append(overlay)

# Overlay masks on original image
overlayed = np.copy(img)

for overlay in overlays:
  # Apply alpha blending
  cv2.addWeighted(overlay, 0.15, overlayed, 0.85, 0, overlayed)

overlayed = cv2.convertScaleAbs(overlayed, alpha=1.5, beta=50)


# Display overlayed image
plt.imshow(overlayed) 

# Create legend with colors and concepts
legend_items = []
for i in range(len(overlays)):
    color = [c/255 for c in colors[i]]
    concept = concepts[i]
    legend_items.append(mpatches.Patch(color=color, label=concept))

plt.legend(handles=legend_items, loc='lower left', bbox_to_anchor=(1.05, 0))
plt.axis('off')
plt.show()

## Note

- This Notebook is a demo to get started with Model Training in Clarifai Platform with Python SDK.
- For better accuracy of the Model, Choose your own data and different Templates and Hyperparams.

## Clarifai Resources

**Website**: [https://www.clarifai.com](https://www.clarifai.com/)

**Demo**: [https://clarifai.com/demo](https://clarifai.com/demo)

**Sign up for a free Account**: [https://clarifai.com/signup](https://clarifai.com/signup)

**Developer Guide**: [https://docs.clarifai.com](https://docs.clarifai.com/)

**Clarifai Community**: [https://clarifai.com/explore](https://clarifai.com/explore)

**Python SDK Docs**: [https://docs.clarifai.com/python-sdk/api-reference](https://docs.clarifai.com/python-sdk/api-reference)

---